# VectorDB Client Tutorial

This notebook demonstrates how to use the Python SDK to interact with the VectorDB server.

## Prerequisites
1. Ensure the VectorDB server is running (`docker-compose up` or `python start.py`).
2. Install the package if not already installed.

In [ ]:
import sys
import os
import time
# Adjust path to import from src if running locally without install
sys.path.append(os.path.abspath("../../src"))

from llm_backend.client import VectorDBClient

## 1. Initialize Client
Connect to the server. Default is `http://localhost:8000`.

In [ ]:
client = VectorDBClient(
    base_url="http://localhost:8000",
    api_key="dev-key-please-change"  # Match your server's VECTOR_API_KEY
)

# Check health
print("Server Health:", client.health())

## 2. Ingest Data (Upsert)
We will upload a few sample documents. The server handles vectorization (Dense + Sparse) automatically.

In [ ]:
collection_name = "tutorial_collection"

documents = [
    {
        "id": "doc_1",
        "content": "Apple released the new iPhone 15 with titanium build.",
        "metadata": {"category": "tech", "year": 2023, "access_level": 1}
    },
    {
        "id": "doc_2",
        "content": "The stock market saw a significant rally in Q4 led by AI stocks.",
        "metadata": {"category": "finance", "year": 2023, "access_level": 1}
    },
    {
        "id": "doc_3",
        "content": "Delicious homemade apple pie recipe with cinnamon.",
        "metadata": {"category": "food", "access_level": 1}
    }
]

print(f"Upserting {len(documents)} docs...")
resp = client.upsert(collection_name, documents)
print("Upsert Response:", resp)

# Allow some time for async indexing in background
time.sleep(2)

## 3. Hybrid Search
Now let's search. The system uses Hybrid Search (Dense + BM25/Splade) + Reranking.

In [ ]:
query = "apple phone"
results = client.search(
    collection_name=collection_name,
    query=query,
    top_k=3
)

print(f"Search results for '{query}':")
for r in results:
    print(f"- [{r['score']:.4f}] {r['text']} (Metadata: {r.get('payload', {})})")

## 4. RBAC (Security)
Test access control by simulating a user with restricted access.

In [ ]:
# Example: User who can only access their own tenant data
user_context = {
    "user_id": "user_123",
    "role": "user"
}

# Search with user context
results_rbac = client.search(
    collection_name=collection_name,
    query="investment",
    user_context=user_context
)

print(f"RBAC Search Results:", len(results_rbac))